# CIDACS: SUPORTE EM ROTINAS DE PRODUÇÃO DE DADOS - Data analysis / Practical Test

This is a practical and technical test whose objective is to evaluate how could you analyze and interprets data. The purpose of this test is to identify your capacity to analyze data and maybe to suggest news insights.

The dataset is related to the Basic Health Unity (UBS), it was disponibility by the Ministry of Health of Brazil, it shows the cities that received the program of the government.

Consider that you are working on a task that is part of delivering a project in production, so it is interesting to document and build well-structured code. Answers, code documentation and comments can be written in Portuguese.

## Challenge
Based on the data provided, answer the following questions:

1. Create a variable with the UF code (Unidade Federativa) where the UBS are located.
    1. What are the UF codes present in the database?
    2. What are the most frequent UF codes (the 5 most frequent) and how often do they occur?
    3. How many cities are there in these 5 most frequent UFs?
    
    Note: The first two digits of the municipality code represent the UF.

2. There is a fictitious date variable (`data_eq`) in the database created only for this selection process. Create a variable with the year extracted from that variable.
    1. Show the distribution of records by year.

3. Create a variable with the length of the phone field
    1. What are the least frequent lengths (the 5 least frequent) of phone numbers?
    2. How many records have a phone length other than 10? What is the percentage of this value considering the size of the database?
    
    Bonus Question: Is there information in the phone variable that does not correspond to a phone number? If so, remove this information.
    
4. How many records achieved an above or much above average performance regarding the status of the drugs? What is the percentage of this value considering the size of the database?

5. How many records in Bahia (code 29) achieved an average performance in relation to the situation of the drugs and a performance above or much above the average in relation to the situation of the equipment? What is the percentage of this value considering the size of the database?

6. How many records had the worst performance in all variables that measure performance situations? What is the percentage of this value considering the size of the database?


## Data dictionary


* cod_munic: `IBGE code, lenght: 7 characters.` 
* dsc_bairro: `Name of neighborhood.`
* dsc_cidade: `Name of city.`
* cod_cnes: `CNES code`
* nom_estab: `Name of the establishment`
* dsc_endereco: `Address of UBS`
* dsc_estrut_fisic_ambiencia: `Situation-related to general structure`
* dsc_adap_defic_fisic_idosos: `Situation-related to adaptations for the disabled and the elderly`
* dsc_equipamentos: `Situation-related to equipament`
* dsc_medicamentos: `Situation-related to medicine`
* data_eq: `Date variable`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

database = pd.read_csv('ubs.csv')

# 1. Create a variable with the UF code (Unidade Federativa) where the UBS are located.

In [ ]:
UF = database['cod_munic'].astype(str)
UF = UF.str[:2]
database['UF'] = UF
display(database)

# What are the UF codes present in the database?

In [ ]:
uf_codes = pd.DataFrame(pd.unique(database['UF']))
uf_codes.columns = ['UF']
display(uf_codes)

# What are the most frequent UF codes (the 5 most frequent) and how often do they occur?

In [ ]:
most_freqUF = pd.DataFrame(pd.value_counts(database['UF']).head(5))
most_freqUF.columns = ['Frequency']
most_freqUF['UF'] = most_freqUF.index
display(most_freqUF)

# How many cities are there in these 5 most frequent UFs?

In [ ]:
#Organizar por UF e contar os valores únicos de municípios
UF_group = database.groupby('UF')['cod_munic'].nunique()
#Usar DF
UF_group = pd.DataFrame(UF_group)
UF_group['UF'] = UF_group.index

#Comparar UF agrupado com UF mais frequente
UF_group.query("UF in %s" % most_freqUF['UF'].values.tolist())


# 2. There is a fictitious date variable (`data_eq`) in the database created only for this selection process. Create a variable with the year extracted from that variable.


In [ ]:
database['year'] = database['data_eq'].str[:4]

# Show the distribution of records by year.

In [ ]:
year_dist = pd.value_counts(database['year']).to_frame()
year_dist.columns = ['records']
year_dist['year'] = year_dist.index
display(year_dist)

year_dist.plot(kind = 'bar', x = 'year', y='records')
plt.ylim(12500, 12700)
plt.show()

# 3. Create a variable with the length of the phone field

In [ ]:
#Bonus Question: Is there information in the phone variable that does not correspond to a phone number? If so, remove this information.
def check_phone_num(phone):
    phone = phone.translate({ord(i): None for i in '()'})
    if phone.isnumeric():	
        return phone
    else:
        return ''

database['dsc_telefone'] = database['dsc_telefone'].apply(check_phone_num)
database['phone_len'] = database['dsc_telefone'].apply(len)


# What are the least frequent lengths (the 5 least frequent) of phone numbers?

In [ ]:
most_freq_phone_len = pd.DataFrame(pd.value_counts(database['phone_len']))
most_freq_phone_len.columns = ['Frequency']
most_freq_phone_len['length'] = most_freq_phone_len.index
most_freq_phone_len = most_freq_phone_len.drop([0], axis = 0)
display(most_freq_phone_len.tail(5))


# How many records have a phone length other than 10? 

In [ ]:
most_freq_phone_len[most_freq_phone_len['length']!=10].count()['length']

# What is the percentage of this value considering the size of the database?

In [ ]:
percent = most_freq_phone_len[most_freq_phone_len['length']!=10].sum()['Frequency']/most_freq_phone_len.sum()['Frequency']*100

print('Percentage of the records that have a phone length other than 10: %.2f%%' % percent)


# 4. How many records achieved an above or much above average performance regarding the status of the drugs? 

In [ ]:
drugs = database.apply(lambda x : True
    if 'acima' in x['dsc_medicamentos'] else False, axis = 1)
good_records = len(drugs[drugs == True].index)
print(good_records)

# What is the percentage of this value considering the size of the database?

In [ ]:
drugs_percent = good_records/len(drugs)*100
print('percentage of above or much above average: %.2f%%' % drugs_percent)

# 5. How many records in Bahia (code 29) achieved an average performance in relation to the situation of the drugs and a performance above or much above the average in relation to the situation of the equipment?

In [ ]:
def situation5(x):
    if x['UF'] == '29' and 'mediano' in x['dsc_medicamentos'] and 'acima' in x['dsc_equipamentos']:
        return True
    else:
        return False

records_5 = database.apply(situation5, axis = 1)
records_5_match = len(records_5[records_5 == True].index)
print('answer: %4d records' % records_5_match)


# What is the percentage of this value considering the size of the database?

In [ ]:
records_5_percent = records_5_match/len(records_5)*100
print('percentage of above or much above average: %.2f%%' % records_5_percent)

# 6. How many records had the worst performance in all variables that measure performance situations?

In [ ]:
def situation6(x):
    if 'mediano' in x['dsc_medicamentos'] and 'mediano' in x['dsc_equipamentos'] and 'mediano' in x['dsc_estrut_fisic_ambiencia'] and 'mediano' in x['dsc_adap_defic_fisic_idosos']:
        return True
    else:
        return False

records_6 = database.apply(situation6, axis = 1)
records_6_match = len(records_6[records_6 == True].index)
print('answer: %4d records' % records_6_match)

# What is the percentage of this value considering the size of the database?

In [ ]:
records_6_percent = records_6_match/len(records_6)*100
print('percentage of worst performance: %.2f%%' % records_6_percent)